In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


#### Create the training data for use with the neural network. We must also save the affine scaling parameters.

In [2]:
from Chempy.parameter import ModelParameters

In [27]:
a = ModelParameters()

# Define training size
N = 8.  # 6 grid points per parameter
widths = a.training_widths # Gaussian training widths for parameters

In [28]:
# Make array for log time
time_array = np.linspace(-0.25,3.,int(N)) # make array wider than observed time dataset

In [29]:
from scipy.stats import norm as gaussian
prob = np.linspace(1/(N+1), 1-1/(N+1), int(N))
grids = [gaussian.ppf(prob) for _ in range(len(a.p0))] # Normalize to unit Gaussian
grids.append(time_array) ## add time array to this grid
norm_grid = np.array(np.meshgrid(*grids)).T.reshape(-1,len(a.p0)+1)

In [30]:
# Create grid in parameter space
full_widths = list(widths)+[1.]
means = list(a.p0)+[0.]
param_grid = [np.asarray(item)*full_widths+means for item in norm_grid]

**Save the two grids to file:**

- ```norm_grid``` is an array of $N^6$ sets of 6d parameter vectors.
- NB: The first five parameters are **normalized**: to get true values multiply by the width and add the mean value.
- The time parameter is **not** normalized (since it is not drawn froma  Gaussian here)


- ```Param_grid``` is of the same form but with the full grid of parameters (with no normalization applied)
- Each element is of the form $(\Theta_1,\Theta_2,\Lambda_1,\Lambda_2,\Lambda_3,T)$ 


- ```means``` gives the prior means for the parameters (for reconstruction from the normalized parameters)
- ```full_widths``` gives the scaling widths to multiply the normalized parameters by to get the true parameters.

NB: we include a 0 mean and 1. width for the time parameter so we can always write true_parameter = norm_parameter x width + means

In [31]:
np.savez('APOGEE Training Data',norm_grid=norm_grid,param_grid=param_grid,full_widths=full_widths,means=means)